## Treniranje CBoW modela

Ova bilježnica dio je [AI za početnike kurikuluma](http://aka.ms/ai-beginners)

U ovom primjeru, pogledat ćemo kako trenirati CBoW jezični model kako bismo dobili vlastiti Word2Vec prostor za ugrađivanje. Koristit ćemo AG News dataset kao izvor teksta.


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Prvo učitajmo naš skup podataka i definirajmo tokenizer i vokabular. Postavit ćemo `vocab_size` na 5000 kako bismo malo ograničili izračune.


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoW model

CBoW uči predviđati riječ na temelju $2N$ susjednih riječi. Na primjer, kada je $N=1$, dobit ćemo sljedeće parove iz rečenice *I like to train networks*: (like, I), (I, like), (to, like), (like, to), (train, to), (to, train), (networks, train), (train, networks). Ovdje je prva riječ susjedna riječ koja se koristi kao ulaz, a druga riječ je ona koju predviđamo.

Kako bismo izgradili mrežu za predviđanje sljedeće riječi, trebamo pružiti susjednu riječ kao ulaz i dobiti broj riječi kao izlaz. Arhitektura CBoW mreže izgleda ovako:

* Ulazna riječ prolazi kroz sloj za ugrađivanje (embedding layer). Ovaj sloj za ugrađivanje bit će naša Word2Vec ugrađenost, stoga ćemo ga definirati zasebno kao varijablu `embedder`. U ovom primjeru koristit ćemo veličinu ugrađivanja = 30, iako biste možda željeli eksperimentirati s većim dimenzijama (pravi Word2Vec ima 300).
* Vektor ugrađivanja zatim se prosljeđuje kroz linearni sloj koji će predvidjeti izlaznu riječ. Stoga ima `vocab_size` neurona.

Za izlaz, ako koristimo `CrossEntropyLoss` kao funkciju gubitka, također ćemo morati pružiti samo brojeve riječi kao očekivane rezultate, bez one-hot kodiranja.


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## Priprema podataka za treniranje

Sada ćemo programirati glavnu funkciju koja će izračunavati CBoW parove riječi iz teksta. Ova funkcija omogućit će nam da odredimo veličinu prozora i vratit će skup parova - ulaznu i izlaznu riječ. Imajte na umu da se ova funkcija može koristiti na riječima, kao i na vektorima/tensorima - što će nam omogućiti kodiranje teksta prije nego što ga proslijedimo funkciji `to_cbow`.


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


Pripremimo skup podataka za treniranje. Proći ćemo kroz sve vijesti, pozvati `to_cbow` kako bismo dobili popis parova riječi, i dodati te parove u `X` i `Y`. Radi uštede vremena, razmotrit ćemo samo prvih 10k vijesti - lako možete ukloniti ovo ograničenje ako imate više vremena za čekanje i želite dobiti bolje ugrađivanja :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

Također ćemo pretvoriti te podatke u jedan skup podataka i kreirati dataloader:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

Također ćemo pretvoriti te podatke u jedan skup podataka i stvoriti učitavač podataka:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

Sada krenimo s pravim treniranjem. Koristit ćemo optimizer `SGD` s prilično visokom stopom učenja. Također možete isprobati i druge optimizere, poput `Adam`. Trenirat ćemo 10 epoha za početak - i možete ponovno pokrenuti ovu ćeliju ako želite još manji gubitak.


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Isprobavanje Word2Vec-a

Za korištenje Word2Vec-a, izdvojimo vektore koji odgovaraju svim riječima u našem vokabularu:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

Pogledajmo, na primjer, kako se riječ **Pariz** kodira u vektor:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


Zanimljivo je koristiti Word2Vec za traženje sinonima. Sljedeća funkcija će vratiti `n` najbližih riječi za dani unos. Kako bismo ih pronašli, izračunavamo normu $|w_i - v|$, gdje je $v$ vektor koji odgovara našoj ulaznoj riječi, a $w_i$ je kodiranje $i$-te riječi u vokabularu. Zatim sortiramo niz i vraćamo odgovarajuće indekse koristeći `argsort`, te uzimamo prvih `n` elemenata liste, koji kodiraju pozicije najbližih riječi u vokabularu.


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## Ključne točke

Koristeći pametne tehnike poput CBoW-a, možemo trenirati Word2Vec model. Također možete pokušati trenirati skip-gram model koji je osposobljen za predviđanje susjedne riječi na temelju središnje, i provjeriti koliko dobro funkcionira.



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden korištenjem AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane stručnjaka. Ne preuzimamo odgovornost za bilo kakve nesporazume ili pogrešne interpretacije proizašle iz korištenja ovog prijevoda.
